In [33]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [34]:
import time
import random
from tqdm import tqdm
import re

In [35]:
s = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=s)
driver.get("https://www.otomoto.pl")



====== WebDriver manager ======
Current firefox version is 95.0
Get LATEST geckodriver version for 95.0 firefox
Driver [/Users/denys_kryzhanivskyi/.wdm/drivers/geckodriver/macos/v0.30.0/geckodriver] found in cache


In [37]:
# close accept privacy
try:
    WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "button#onetrust-accept-btn-handler"))).click()
except:
    pass

# Retrieve dropdown values

In [38]:
def get_dropdown_list(input_id='filter_enum_make'):
    try:
        # Selecting input field
        input_field = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, f"input#{input_id}")))
        
        # Clearing input field and revealing dropdown
        input_field.clear()
        time.sleep(1)
        input_field.click()
                
        dropdown = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.ds-dropdown-wrapper")))
                
        # Retrieving all names of dropdown list
        dropdown_spans = WebDriverWait(dropdown, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "span")))
        
        values = [span.text for span in dropdown_spans]
        values_cleaned = sorted(list({re.sub('\(\d+\)', '', v).strip() for v in values}))
        
        # Make input unfocused
        driver.execute_script("arguments[0].blur();", input_field)
        
        return values_cleaned
    except Exception as e:
#         print(e)
        return []
        

# Clear input

In [91]:
def clear_input(input_field):
    driver.execute_script("arguments[0].focus();", input_field)
    input_field.send_keys(Keys.COMMAND + 'a')
    input_field.send_keys(Keys.DELETE)
    time.sleep(0.5)
    driver.execute_script("arguments[0].blur();", input_field)

In [120]:
brands = get_dropdown_list()[5:]
brands

['Aro',
 'Asia',
 'Aston Martin',
 'Audi',
 'Austin',
 'Autobianchi',
 'BAC',
 'BMW',
 'BMW-ALPINA',
 'Baic',
 'Befard',
 'Bentley',
 'Bobcat',
 'Bolloré',
 'Brilliance',
 'Bugatti',
 'Buick',
 'Cadillac',
 'Casalini',
 'Caterham',
 'Chatenet',
 'Chevrolet',
 'Chrysler',
 'Citroën',
 'Comarth',
 'Cupra',
 'DFSK',
 'DKW',
 'DR MOTOR',
 'DS Automobiles',
 'Dacia',
 'Daewoo',
 'Daihatsu',
 'De Lorean',
 'Dodge',
 'Eagle',
 'FAW',
 'Ferrari',
 'Fiat',
 'Ford',
 'GMC',
 'GWM',
 'Galloper',
 'Gaspardo',
 'Gaz',
 'Geely',
 'Gonow',
 'Grecav',
 'Hammer',
 'Holden',
 'Honda',
 'Hummer',
 'Hyundai',
 'Infiniti',
 'Inny',
 'Isuzu',
 'Iveco',
 'Jaguar',
 'Jeep',
 'John Deere',
 'Kaipan',
 'Kia',
 'Kuhn',
 'LEVC',
 'LTI',
 'Lada',
 'Lamborghini',
 'Lancia',
 'Land Rover',
 'Lexus',
 'Ligier',
 'Lincoln',
 'Lotus',
 'LuAZ',
 'MAN',
 'MG',
 'MINI',
 'Mahindra',
 'Maruti',
 'Maserati',
 'Maybach',
 'Mazda',
 'McLaren',
 'Mercedes-Benz',
 'Mercury',
 'Microcar',
 'Mitsubishi',
 'Morgan',
 'Moskwicz',
 

# Create dict brand-model-gen

In [121]:
brands_models_generations={}

In [122]:
def scrap_brands_models_generation(brands=brands):
    
    # Selecting brand input
    input_brand = driver.find_element(By.CSS_SELECTOR, "input#filter_enum_make")
    
    for brand in tqdm(brands):
        brands_models_generations[brand]=[]
        
        # Providing brand name
        input_brand.send_keys(brand)
        input_brand.send_keys(Keys.RETURN)
        
        time.sleep(1)
        
        # Selecting model input
        input_model = driver.find_element(By.CSS_SELECTOR, "input#filter_enum_model")
        
        input_model.click()
        time.sleep(0.5)
        
        models = get_dropdown_list('filter_enum_model')
        
        if len(models) == 0:
            print(f'{brand} - {[]}')
            brands_models_generations[brand].append({})
            time.sleep(0.2)
            clear_input(input_brand)
            continue
    
        print(f'{brand} - {models}')

        for model in tqdm(models):
            input_model.send_keys(model)
            input_model.send_keys(Keys.RETURN)
            time.sleep(1)

            input_generation = driver.find_element(by='xpath',value='//*[@id="filter_enum_generation"]')
            input_generation.click()
            time.sleep(0.5)
            
            generations = get_dropdown_list('filter_enum_generation')
            
            if len(generations) == 0:
                brands_models_generations[brand].append({model: None})
                clear_input(input_model)
                continue
                
            brands_models_generations[brand].append({model:generations})
            if input_generation.is_enabled():
                clear_input(input_model)
        
        driver.execute_script("arguments[0].focus();", input_brand)
        clear_input(input_brand)

In [ ]:
scrap_brands_models_generation()

  0%|          | 0/27 [00:00<?, ?it/s]

Aro - ['145', '146', '147', '155', '156', '159', '164', '166', '33', '4C', '75', '90', 'Alfasud', 'Alfetta', 'Brera', 'Crosswagon', 'GT', 'GTV', 'Giulia', 'Giulietta', 'Inny', 'Mito', 'RS', 'Spider', 'Sportwagon', 'Sprint', 'Stelvio']



  0%|          | 0/1 [00:00<?, ?it/s]

Asia - ['Rocsta']



  0%|          | 0/28 [00:00<?, ?it/s]

...


  0%|          | 0/8 [00:00<?, ?it/s]

Syrena - ['101', '102', '103', '104', '105', 'Bosto', 'Inny', 'R-20']



  0%|          | 0/8 [00:00<?, ?it/s]

TVR - ['Cerbera', 'Chimaera', 'Griffith', 'Inny', 'Tamora', 'Tasmin', 'Tuscan', 'Vixen']



 62%|██████▎   | 5/8 [00:14<00:08,  2.87s/it]

In [124]:
brands_models_generations

{'Aro': [{'145': None},
  {'146': None},
  {'147': None},
  {'155': None},
  {'156': None},
  {'159': None},
  {'164': None},
  {'166': None},
  {'33': None},
  {'4C': None},
  {'75': None},
  {'90': None},
  {'Alfasud': None},
  {'Alfetta': None},
  {'Brera': None},
  {'Crosswagon': None},
  {'GT': None},
  {'GTV': None},
  {'Giulia': None},
  {'Giulietta': None},
  {'Inny': None},
  {'Mito': None},
  {'RS': None},
  {'Spider': ['SZ']},
  {'Sportwagon': None},
  {'Sprint': None},
  {'Stelvio': None}],
 'Asia': [{'Rocsta': None}],
 'Aston Martin': [{'Bulldog': None},
  {'Cygnet': None},
  {'DB': None},
  {'DB Mk III': None},
  {'DB1': None},
  {'DB11': None},
  {'DB2': ['4']},
  {'DB4': None},
  {'DB5': None},
  {'DB6': None},
  {'DB7': None},
  {'DB9': None},
  {'DBS': None},
  {'DBS Superleggera': None},
  {'DBX': None},
  {'Inny': None},
  {'Lagonda': None},
  {'One-77': None},
  {'Rapide': None},
  {'V12 Vanquish': None},
  {'V12 Vantage': None},
  {'V8': None},
  {'V8 Vantage': No

In [125]:
import json
with open('data.json', 'w') as fp:
    json.dump(brands_models_generations, fp)